In [ ]:
!pip install PyWavelets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Traditional methods for denoising

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pywt
from skimage.metrics import structural_similarity as ssim

# Define utility functions for metrics
def calculate_psnr(original, denoised):
    mse_value = np.mean((original - denoised) ** 2)
    if mse_value == 0:
        return 100  # Avoid log(0)
    max_pixel = 255.0
    return 10 * np.log10(max_pixel ** 2 / mse_value)

def calculate_ssim(original, denoised):
    return ssim(original, denoised, data_range=original.max() - original.min())

def calculate_mse(original, denoised):
    return np.mean((original - denoised) ** 2)

# Define denoising methods
def apply_denoising(image, method):
    if method == "Mean":
        return cv2.blur(image, (5, 5))
    elif method == "Median":
        return cv2.medianBlur(image, 5)
    elif method == "Gaussian":
        return cv2.GaussianBlur(image, (5, 5), 1.5)
    elif method == "Bilateral":
        return cv2.bilateralFilter(image, 9, 75, 75)
    elif method == "Weiner":
        return wiener_filter(image)
    elif method == "Wavelet":
        return wavelet_denoise(image)
    elif method == "Fourier":
        return fourier_denoise(image)
    else:
        raise ValueError("Unknown method")

# Wiener filter
def wiener_filter(image):
    from scipy.signal import wiener
    filtered = wiener(image, (5, 5))
    filtered = np.nan_to_num(filtered, nan=0.0)
    return np.clip(filtered, 0, 255).astype(np.uint8)

# Wavelet denoising
def wavelet_denoise(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    coeffs_thresh = [coeffs[0]]
    for details in coeffs[1:]:
        coeffs_thresh.append(tuple(pywt.threshold(c, value=30, mode='soft') for c in details))
    return pywt.waverec2(coeffs_thresh, 'haar').astype(np.uint8)

# Fourier denoising
def fourier_denoise(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    mask[crow-30:crow+30, ccol-30:ccol+30] = 1
    fshift *= mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    return np.abs(img_back).astype(np.uint8)

# Base path to the dataset
base_path = '/content/drive/MyDrive/CS415Project/Noisy COVID-19 X-ray Dataset'

# Folders for the noise types
main_folders = ["Basic", "AWGN", "Noisy"]

# Define the category folders within each main folder
categories = ["covid", "normal", "viral"]

# Define methods and metrics
methods = ["Mean", "Median", "Gaussian", "Bilateral", "Weiner", "Wavelet", "Fourier"]
metrics = {"PSNR": calculate_psnr, "SSIM": calculate_ssim, "MSE": calculate_mse}

# Initialize results dictionary
results = {folder: {category: {method: {metric: [] for metric in metrics} for method in methods}
                    for category in categories} for folder in main_folders}

# Loop through each main folder, category, and method
for main_folder in main_folders:
    for category in categories:
        category_path = os.path.join(base_path, main_folder, "test", category)
        image_files = os.listdir(category_path)[:50]  # Process the first 50 images

        for img_name in image_files:
            img = cv2.imread(os.path.join(category_path, img_name), cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Warning: Image {img_name} in {category_path} could not be loaded.")
                continue

            for method in methods:
                denoised_img = apply_denoising(img, method)
                for metric, func in metrics.items():
                    results[main_folder][category][method][metric].append(func(img, denoised_img))

# Average results
average_results = {main_folder: {category: {method: {metric: np.mean(values)
                                                     for metric, values in metrics.items() }
                                           for method, metrics in category_dict.items() }
                                 for category, category_dict in main_dict.items() }
                   for main_folder, main_dict in results.items()}

# Prepare data for Excel and plotting
data = []
for main_folder in main_folders:
    for category in categories:
        for method in methods:
            row = {'Noise Type': main_folder, 'Category': category, 'Method': method}
            for metric in metrics:
                row[metric] = average_results[main_folder][category][method][metric]
            data.append(row)

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel file
output_path = os.path.join(base_path, 'denoising_metrics_results.xlsx')
df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

# Create bar charts for each category
chart_base_path = os.path.join(base_path, "charts")
os.makedirs(chart_base_path, exist_ok=True)

# Generate bar charts for each category
for noise_type in ["AWGN", "Noisy"]:
    for category in categories:
        # Filter data for current category and noise type
        filtered_data = df[(df['Noise Type'] == noise_type) & (df['Category'] == category)]

        # Plot for each metric (PSNR, MSE, SSIM)
        for metric in metrics:
            fig, ax = plt.subplots(figsize=(8, 5))

            # Extract values for each method
            method_values = [filtered_data[filtered_data['Method'] == method][metric].values[0] for method in methods]

            # Plot the bar chart
            ax.bar(methods, method_values, color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2"])

            # Set chart labels and title
            ax.set_xlabel("Methods")
            ax.set_ylabel(f"{metric} Value")
            ax.set_title(f"{metric} - {noise_type} - {category.capitalize()}")

            # Add the metric value on top of each bar
            for i, v in enumerate(method_values):
                ax.text(i, v + 0.5, round(v, 2), ha='center')

            # Save chart
            chart_path = os.path.join(chart_base_path, f"{noise_type}_{category}_{metric}.png")
            plt.savefig(chart_path)
            plt.close(fig)

print(f"Charts saved in {chart_base_path}")


Results saved to /content/drive/MyDrive/CS415Project/Noisy COVID-19 X-ray Dataset/denoising_metrics_results.xlsx
Charts saved in /content/drive/MyDrive/CS415Project/Noisy COVID-19 X-ray Dataset/charts
